In [ ]:
import pandas as pd
#cargar csv imagenes
df= pd.read_csv("/Users/n.arcos89/Desktop/Bootcamp_Data/DIVA_DIGITAL_Proyecto Final/Data/publicaciones_pixabay.csv")
#reemplazar la palabra POST por IMG en columnas Imagen y Ruta
df["Imagen"] = df["Imagen"].str.replace("POST", "IMG")
df["Ruta"] = df["Ruta"].str.replace("POST", "IMG")
#comprobar
print(df.head())

    id_post       Fecha       Imagen  \
0   POST_98  2024-08-05   IMG_98.jpg   
1  POST_190  2024-08-08  IMG_190.jpg   
2   POST_38  2024-08-12   IMG_38.jpg   
3   POST_73  2024-08-14   IMG_73.jpg   
4   POST_44  2024-08-16   IMG_44.jpg   

                                                Ruta  
0  /Users/n.arcos89/Desktop/Bootcamp_Data/DIVA_DI...  
1  /Users/n.arcos89/Desktop/Bootcamp_Data/DIVA_DI...  
2  /Users/n.arcos89/Desktop/Bootcamp_Data/DIVA_DI...  
3  /Users/n.arcos89/Desktop/Bootcamp_Data/DIVA_DI...  
4  /Users/n.arcos89/Desktop/Bootcamp_Data/DIVA_DI...  


In [ ]:
#reemplazar rutas de imagenes

In [6]:
#descargar nuevo df resultante con el nombre publicaciones_pixabay_ok.csv
df.to_csv("/Users/n.arcos89/Desktop/Bootcamp_Data/DIVA_DIGITAL_Proyecto Final/Data/publicaciones_pixabay_ok.csv", index=False)
print("Archivo actualizado y descargado correctamente.")

Archivo actualizado y descargado correctamente.


In [9]:
import pandas as pd
import os

print("🔍 SOLUCIÓN AL PROBLEMA DE SEPARADORES")
print("=" * 50)

# Cargar archivos con separadores correctos
try:
    print("📂 Cargando data_demo.csv...")
    df_demo = pd.read_csv('/Users/n.arcos89/Desktop/Bootcamp_Data/DIVA_DIGITAL_Proyecto Final/Data/data_demo.csv', sep=';')
    print(f"✅ data_demo_ok cargado: {len(df_demo)} filas, {len(df_demo.columns)} columnas")
    print("📋 Columnas:", df_demo.columns.tolist())
    
except Exception as e:
    print(f"❌ Error: {e}")

try:
    print("\n📂 Cargando publicaciones_pixabay_ok.csv...")
    df_pixabay = pd.read_csv('/Users/n.arcos89/Desktop/Bootcamp_Data/DIVA_DIGITAL_Proyecto Final/Data/publicaciones_pixabay_ok.csv', sep=',')
    print(f"✅ publicaciones_pixabay_ok cargado: {len(df_pixabay)} filas, {len(df_pixabay.columns)} columnas")
    print("📋 Columnas:", df_pixabay.columns.tolist())
    
except Exception as e:
    print(f"❌ Error: {e}")

# Limpiar nombres de columnas por si acaso
df_demo.columns = df_demo.columns.str.strip()
df_pixabay.columns = df_pixabay.columns.str.strip()

# Verificar columnas id_post
print(f"\n🔍 VERIFICACIÓN:")
print(f"¿Existe 'id_post' en df_demo? {'id_post' in df_demo.columns}")
print(f"¿Existe 'id_post' en df_pixabay? {'id_post' in df_pixabay.columns}")

# Identificar columnas comunes antes del merge
common_columns = set(df_demo.columns) & set(df_pixabay.columns)
print(f"📊 Columnas comunes: {list(common_columns)}")

if 'id_post' in df_demo.columns and 'id_post' in df_pixabay.columns:
    print("\n✅ ¡AMBAS COLUMNAS ENCONTRADAS!")
    
    # Mostrar algunos valores para verificar compatibilidad
    print(f"Valores de ejemplo en demo: {df_demo['id_post'].head().tolist()}")
    print(f"Valores de ejemplo en pixabay: {df_pixabay['id_post'].head().tolist()}")
    
    # Verificar si hay coincidencias antes del merge
    common_ids = set(df_demo['id_post']) & set(df_pixabay['id_post'])
    print(f"📊 IDs comunes encontrados: {len(common_ids)}")
    if len(common_ids) > 0:
        print(f"Ejemplos de IDs comunes: {list(common_ids)[:5]}")
    else:
        print("⚠️ No hay IDs comunes entre los archivos")
    
    # Hacer el merge con sufijos explícitos para columnas duplicadas
    print("\n🔗 REALIZANDO MERGE...")
    result = pd.merge(df_demo, df_pixabay, on='id_post', how='left', suffixes=('_demo', '_pixabay'))
    
    print(f"✅ Merge exitoso: {len(result)} filas, {len(result.columns)} columnas")
    
    # Mostrar todas las columnas del resultado para ver qué hay disponible
    print(f"📋 Columnas en el resultado: {result.columns.tolist()}")
    
    # Verificar cuántas filas se mergearon exitosamente
    # Buscar columnas que vengan solo del archivo pixabay
    pixabay_only_cols = []
    for col in df_pixabay.columns:
        if col != 'id_post':  # Excluir la columna de merge
            if col in df_demo.columns:
                # Si la columna existe en ambos, buscar la versión _pixabay
                pixabay_col_name = f"{col}_pixabay"
                if pixabay_col_name in result.columns:
                    pixabay_only_cols.append(pixabay_col_name)
            else:
                # Si la columna solo existe en pixabay
                if col in result.columns:
                    pixabay_only_cols.append(col)
    
    print(f"📋 Columnas de pixabay en resultado: {pixabay_only_cols}")
    
    if pixabay_only_cols:
        merged_count = result[pixabay_only_cols[0]].notna().sum()
        print(f"📊 Filas con datos de pixabay: {merged_count}")
        
        # Buscar la columna de imagen (puede tener diferentes nombres)
        imagen_cols = [col for col in result.columns if 'imagen' in col.lower()]
        if imagen_cols:
            imagen_col = imagen_cols[0]
            imagen_count = result[imagen_col].notna().sum()
            print(f"📊 Filas con imágenes (columna '{imagen_col}'): {imagen_count}")
        else:
            print("⚠️ No se encontró columna de imagen")
    
    # Limpiar el resultado - mantener solo una versión de las columnas duplicadas
    print("\n🧹 LIMPIANDO COLUMNAS DUPLICADAS...")
    result_clean = result.copy()
    
    # Para cada columna común, decidir cuál mantener
    for col in common_columns:
        if col != 'id_post':  # No tocar la columna de merge
            demo_col = f"{col}_demo"
            pixabay_col = f"{col}_pixabay"
            
            if demo_col in result_clean.columns and pixabay_col in result_clean.columns:
                # Combinar: usar pixabay si está disponible, sino demo
                result_clean[col] = result_clean[pixabay_col].fillna(result_clean[demo_col])
                # Eliminar las columnas con sufijos
                result_clean = result_clean.drop([demo_col, pixabay_col], axis=1)
                print(f"✅ Columna '{col}' combinada")
    
    print(f"📋 Columnas después de limpiar: {result_clean.columns.tolist()}")
    
    # Definir la ruta completa para guardar el archivo
    output_path = '/Users/n.arcos89/Desktop/Bootcamp_Data/DIVA_DIGITAL_Proyecto Final/Data/data_unificada.csv'
    
    # Verificar que el directorio existe
    output_dir = os.path.dirname(output_path)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"📁 Directorio creado: {output_dir}")
    
    # Guardar resultado limpio en la ruta especificada
    result_clean.to_csv(output_path, index=False, sep=';')
    print(f"✅ Archivo guardado en: {output_path}")
    
    # Verificar que el archivo se guardó correctamente
    if os.path.exists(output_path):
        file_size = os.path.getsize(output_path)
        print(f"📁 Archivo verificado: {file_size} bytes")
    else:
        print("❌ Error: El archivo no se guardó correctamente")
    
    # Mostrar algunas filas de ejemplo con columnas disponibles
    print("\n📊 EJEMPLO DEL RESULTADO:")
    try:
        # Intentar mostrar con columnas específicas que deberían existir
        cols_to_show = ['id_post']
        
        # Añadir columnas importantes si existen
        important_cols = ['Fecha', 'Imagen', 'Ruta', 'Canal', 'Formato', 'Alcance']
        for col in important_cols:
            if col in result_clean.columns:
                cols_to_show.append(col)
        
        # Mostrar máximo 6 columnas
        cols_to_show = cols_to_show[:6]
        print(f"Columnas a mostrar: {cols_to_show}")
        print(result_clean[cols_to_show].head())
        
    except Exception as e:
        print(f"Error mostrando ejemplo: {e}")
        print("Mostrando primeras 5 columnas:")
        print(result_clean.iloc[:5, :5])
    
    # Estadísticas finales
    print(f"\n📈 ESTADÍSTICAS DEL MERGE:")
    print(f"- Filas originales en demo: {len(df_demo)}")
    print(f"- Filas en pixabay: {len(df_pixabay)}")
    print(f"- Filas en resultado: {len(result_clean)}")
    
    if pixabay_only_cols:
        # Buscar la primera columna de pixabay en el resultado limpio
        pixabay_col_clean = None
        for col in ['Imagen', 'Ruta']:
            if col in result_clean.columns:
                pixabay_col_clean = col
                break
        
        if pixabay_col_clean:
            merged_rows = result_clean[pixabay_col_clean].notna().sum()
            print(f"- Filas que se mergearon exitosamente: {merged_rows}")
            print(f"- Porcentaje de merge exitoso: {merged_rows/len(result_clean)*100:.1f}%")
    
else:
    print("❌ Aún no se encuentran las columnas")
    print(f"Columnas en demo: {df_demo.columns.tolist()}")
    print(f"Columnas en pixabay: {df_pixabay.columns.tolist()}")

print(f"\n📊 RESUMEN FINAL:")
print(f"data_demo_ok shape: {df_demo.shape}")
print(f"publicaciones_pixabay_ok shape: {df_pixabay.shape}")
if 'result_clean' in locals():
    print(f"Resultado final shape: {result_clean.shape}")
    print(f"📁 Archivo guardado en: /Users/n.arcos89/Desktop/Bootcamp_Data/DIVA_DIGITAL_Proyecto Final/Data/data_unificada.csv")
    

🔍 SOLUCIÓN AL PROBLEMA DE SEPARADORES
📂 Cargando data_demo.csv...
✅ data_demo_ok cargado: 200 filas, 24 columnas
📋 Columnas: ['id_post', 'Fecha', 'Hora', 'Canal', 'Formato', 'Imagen', 'Alcance', 'Interacciones', 'Reproducciones', 'Duracion_video', 'DM', 'Retencion', 'Visitas_perfil', 'Link_bio', 'Visitas_producto', 'Carrito', 'Compras', 'Valor_compra', 'Contacto', 'Inversion', 'ROI', 'Engagement', 'CPC', 'Unnamed: 23']

📂 Cargando publicaciones_pixabay_ok.csv...
✅ publicaciones_pixabay_ok cargado: 200 filas, 4 columnas
📋 Columnas: ['id_post', 'Fecha', 'Imagen', 'Ruta']

🔍 VERIFICACIÓN:
¿Existe 'id_post' en df_demo? True
¿Existe 'id_post' en df_pixabay? True
📊 Columnas comunes: ['Fecha', 'id_post', 'Imagen']

✅ ¡AMBAS COLUMNAS ENCONTRADAS!
Valores de ejemplo en demo: ['POST_190', 'POST_146', 'POST_74', 'POST_183', 'POST_198']
Valores de ejemplo en pixabay: ['POST_98', 'POST_190', 'POST_38', 'POST_73', 'POST_44']
📊 IDs comunes encontrados: 131
Ejemplos de IDs comunes: ['POST_125', 'POST_